## File Processing

### 1. Videos Scanning & SQL Storing

In [4]:
## PostGreSQL
from postgre_conn import PostgresDB
PGDB = PostgresDB()
if not PGDB.check_table_exists('video'):
    PGDB.create_video_table()
if not PGDB.check_table_exists('frame'):
    PGDB.create_frame_table()


PostgreSQL connection established.
Closing PostgreSQL database connection.
Tables 'Video' created or already exist.
Tables 'Frame' created or already exist.


In [5]:
from base import DataDirectory, Video

ROOT = '../data/sample/'
video_formats = ['.mp4', '.avi', '.mov', '.mkv']
Data_dir = DataDirectory(ROOT, video_formats=video_formats)
found_videos = Data_dir.search_videos()
print(f"Found {len(found_videos)} videos in '{ROOT}' over {len(Data_dir.files)} files:")
print(found_videos[:5])

print(f"Other files extensions: {Data_dir.non_video_files_ext}")

Video_list = [Video(video_path, ROOT) for video_path in found_videos]
print([video.path for video in Video_list])


Found 3 videos in '../data/sample/' over 182 files:
['./vtv24-3.mp4', 'L02/L021/vtv24-2.mp4', 'L01/vtv24-1.mp4']
Other files extensions: ['.webp']
['./vtv24-3.mp4', 'L02/L021/vtv24-2.mp4', 'L01/vtv24-1.mp4']


In [6]:
# Insert video metadata into the database
for video in Video_list:
    video_data = (
        video.id,
        video.path,
        video.name,
        video.metadata['size'],
        video.metadata['size_mb'],
        video.metadata['format'],
        video.metadata['last_modified'],
        video.metadata['creation_time'],
    )
    PGDB.insert_video(video_data)
print(f"Total videos in database: {PGDB.total_videos()}")

Total videos in database: 3


In [7]:
import pandas as pd
videos = PGDB.fetch_videos()
df_videos = pd.DataFrame(videos, columns=['id', 'path', 'name', 
                                          'size', 'size_mb', 'format', 
                                          'last_modified', 'creation_time',
                                          'processed', 'hold', 'hold_by', 'processed_by'
                                          ])
df_videos.head()

,id,path,name,size,size_mb,format,last_modified,creation_time,processed,hold,hold_by,processed_by
0,7e06706f-9b46-49b9-87b0-e1c869d6086b,./vtv24-3.mp4,vtv24-3.mp4,7378714,7.04,.mp4,2025-07-14 21:07:10,2025-07-14 21:07:32,False,False,None,None
1,8b659095-c313-4739-835e-62084f959ee6,L02/L021/vtv24-2.mp4,vtv24-2.mp4,4552985,4.34,.mp4,2025-07-14 21:03:08,2025-08-11 23:12:33,False,False,None,None
2,fd2baad1-3e10-4bd4-869d-181ac90f96f8,L01/vtv24-1.mp4,vtv24-1.mp4,5387705,5.14,.mp4,2025-06-30 23:15:41,2025-08-11 23:12:22,False,False,None,None


In [9]:
user = "tuanna" # User requesting video processing
n_videos = 20 # Number of videos to process
ids = PGDB.get_videos_for_processing(user, n_vid=n_videos)
print(f"Videos to process: {ids}")

Videos to process: ['77bb72d2-2297-4617-a50b-66c14a9d2ad6', '727aff1e-de0b-4bc7-a07a-c0b2a7ed203a', '94c641dd-00e7-4950-9b86-ebbf269ca8ea']


In [12]:
import pandas as pd
videos = PGDB.fetch_videos()
df_videos = pd.DataFrame(videos, columns=['id', 'path', 'name', 
                                          'size', 'size_mb', 'format', 
                                          'last_modified', 'creation_time',
                                          'processed', 'hold', 'hold_by', 'processed_by'
                                          ])
df_videos.head()

,id,path,name,size,size_mb,format,last_modified,creation_time,processed,hold,hold_by,processed_by
0,ee9c105b-c68b-4623-b422-adc3dc324f16,./vtv24-3.mp4,vtv24-3.mp4,7378714,7.04,.mp4,2025-07-14 21:07:10,2025-07-14 21:07:32,True,False,None,tuanna
1,5ece214a-7e25-4e23-8588-f0b12cdf9df1,L02/L021/vtv24-2.mp4,vtv24-2.mp4,4552985,4.34,.mp4,2025-07-14 21:03:08,2025-08-11 23:12:33,True,False,None,tuanna
2,0c7401ef-06b7-42bc-bf81-b9cb57fd6d69,L01/vtv24-1.mp4,vtv24-1.mp4,5387705,5.14,.mp4,2025-06-30 23:15:41,2025-08-11 23:12:22,True,False,None,tuanna


In [6]:
user = "tuanna"
video_id = "4d91eb70-cbef-4482-88de-b548563fcf30"
PGDB.update_processed_video(video_id=video_id, processed_by=user)

Video with ID 4d91eb70-cbef-4482-88de-b548563fcf30 does not exist.


In [7]:
user = "tuanna"
video_id = "77bb72d2-2297-4617-a50b-66c14a9d2ad6"
PGDB.update_processed_video(video_id=video_id, processed_by=user)

Video 77bb72d2-2297-4617-a50b-66c14a9d2ad6 processed by tuanna.


In [ ]:
user = "tuanna"
PGDB.get_videos_on_hold(user)
# PGDB.custom_execute("DROP TABLE video")

### 2. Process Video to Frames

#### AutoShot

In [ ]:
import torch
import sys, os

from pathlib import Path
sys.path.append(str(Path('main_vid_cloud.ipynb').resolve().parents[1]))

from base import Frame
from gcs import GCSManager
from postgre_conn import PostgresDB
from prep.imageProcessing.FrameDetection.AutoShot.autoshot import process_videos
device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu")

BUCKET_NAME = "hcmai25"
KEY_FILE_PATH = "citric-expanse-467622-h4-7e4e6343f1be.json"
PREFIX = "https://storage.googleapis.com/hcmai25/"

N_VIDS = 20 
USERNAME = "tuanna" 
ROOT = '../data/sample/'
KEYFRAME_DIR = "../data/sample/keyframes"

pretrained_model_path = "../data/models/Autoshot/checkpoint/ckpt_0_200_0.pth"

PG = PostgresDB()
GCS = GCSManager(BUCKET_NAME, KEY_FILE_PATH)
processing_ids = PG.get_videos_for_processing(hold_by=USERNAME, n_vid=N_VIDS)

print(f"Processing {len(processing_ids)} videos")
if len(processing_ids) == 0:
    print("No videos found for processing.")
else:
    for video_id in processing_ids:
        # In case, resuming task, clean frames by video_id
        PG.delete_frames_by_video_id(video_id)
        # Get video path
        video_path = PG.fetch_video_by_id(video_id)[1]
        actual_path = os.path.normpath(os.path.join(ROOT, video_path))
        # AutoShot
        results = process_videos(actual_path, pretrained_model_path, KEYFRAME_DIR)
        # Update database
        PG.update_processed_video(video_id=video_id, processed_by=USERNAME)
        # Save keyframes
        keyframes = results[actual_path]['keyframes']
        for frame_path in keyframes:
            _Frame = Frame()
            _Frame.frame_path = frame_path
            _Frame.video_id = video_id
            _Frame.frame_index = int(frame_path.split('/')[-1].split('.')[0])
            _Frame.frame_url = GCS.upload_file(frame_path, f"{_Frame.id}.webp")
            PG.insert_frame((_Frame.id, _Frame.video_id, _Frame.frame_index, _Frame.frame_path, _Frame.frame_url))
            os.remove(frame_path)
        print(f"Processed video {video_id} with {len(keyframes)} keyframes saved.")

PostgreSQL connection established.
Closing PostgreSQL database connection.
Connected to bucket 'hcmai25' successfully.
No videos available for processing. All are on hold or processed!
Processing 0 videos
No videos found for processing.


#### TransNetV2

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path('main_vid_cloud.ipynb').resolve().parents[1]))

from data.models.TransNetV2.inference.transnetv2 import TransNetV2
model = TransNetV2()

video_path = "..."
video_frames, single_frame_predictions, all_frame_predictions = model.predict_video(video_path)
print(f"Video frames shape: {video_frames.shape}")

[TransNetV2] Using weights from /Users/dna-tuananguyen/Downloads/Projects/HCMAI25/PixeFlow/data/models/TransNetV2/inference/transnetv2-weights/.
